# Stack Overflow Tags

This notebook shows how to train dense vectors for question tags. See [this README](./data/stackoverflow/README.md) for more information about the dataset.

In [1]:
import numpy as np

import pandas as pd

import umap

from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.io import output_notebook

from itembed import (
    pack_itemsets,
    initialize_syn,
    UnsupervisedTask,
    train,
    normalize,
)

In [2]:
# Initialize Bokeh
output_notebook()

Loading BokehJS ...

In [3]:
# Make it reproducible
np.random.seed(42)

In [4]:
# Load raw dataset
tag_df = pd.read_csv("./data/stackoverflow/tags.csv")
tag_df.head(10)

,id,tags
0,4,c#;floating-point;type-conversion;double;decimal
1,11,c#;datetime;time;datediff;relative-time-span
2,13,html;browser;timezone;user-agent;timezone-offset
3,16,c#;linq;web-services;.net-3.5
4,17,mysql;database;binary-data;data-storage
5,19,performance;algorithm;language-agnostic;unix;pi
6,25,c++;c;sockets;mainframe;zos
7,36,sql;sql-server;datatable;rdbms
8,39,c#;.net;vb.net;timer
9,42,php;plugins;architecture;hook


In [5]:
# Get tags as a list of list of string
itemsets = tag_df.tags.str.split(";").values

In [6]:
# Pack itemsets into contiguous arrays
labels, indices, offsets = pack_itemsets(itemsets, min_count=10)
num_label = len(labels)

In [7]:
# Initialize embeddings sets from uniform distribution
num_dimension = 64
syn0 = initialize_syn(num_label, num_dimension)
syn1 = initialize_syn(num_label, num_dimension)

In [8]:
# Define unsupervised task, i.e. using co-occurrences
task = UnsupervisedTask(indices, offsets, syn0, syn1, num_negative=5)

In [9]:
# Do training
# Note: due to a different sampling strategy, more epochs than word2vec are needed
train(task, num_epoch=100)

100%|██████████████████████████████████████████████████████████████████████| 1562400/1562400 [05:50<00:00, 4457.94it/s]


In [10]:
# Both embedding sets are equivalent, just choose one of them
syn = syn0

In [11]:
# Project with UMAP, using cosine similarity measure
model = umap.UMAP(metric="cosine", verbose=1)
projection = model.fit_transform(syn)

UMAP(angular_rp_forest=True, dens_frac=0.0, dens_lambda=0.0, metric='cosine',
     verbose=1)
Construct fuzzy simplicial set
Wed Jun  9 18:38:39 2021 Finding Nearest Neighbors
Wed Jun  9 18:38:39 2021 Building RP forest with 11 trees
Wed Jun  9 18:38:40 2021 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	Stopping threshold met -- exiting after 5 iterations
Wed Jun  9 18:38:55 2021 Finished Nearest Neighbor Search
Wed Jun  9 18:38:58 2021 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Wed Jun  9 18:39:05 2021 Finished embedding


In [12]:
# Pack as a Bokeh data source
source = ColumnDataSource(data=dict(
    x=projection[:, 0],
    y=projection[:, 1],
    label=labels,
))

# Create plot
p = figure(
    width=900,
    height=600,
    tooltips=[
        ("label", "@label"),
    ],
)

# Draw tags as points
p.scatter(
    "x", "y",
    source=source,
)

# Show in notebook
show(p)

In [13]:
# Cosine similarity is equivalent to dot product with normalized vectors
syn_normalized = normalize(syn)

In [14]:
# Closest tags to apples
i = labels.index("word")
similarities = syn_normalized @ syn_normalized[i]
for j in np.argsort(-similarities)[:10]:
    print("#{} {}: {}".format(j, labels[j], similarities[j]))

#2679 word: 0.9999998807907104
#7988 punctuation: 0.7538751363754272
#13959 phrases: 0.7463470697402954
#749 character: 0.7437165379524231
#11359 sentence: 0.7266198396682739
#7991 chars: 0.7258648872375488
#4274 words: 0.7252897024154663
#11833 phrase: 0.7053873538970947
#9693 string-split: 0.7010754346847534
#12497 non-latin: 0.6950054168701172
